In [14]:
from functools import partial
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_columns', None)

In [2]:
raw_file = '../data/raw.csv'
results_file = '../data/results.csv'

In [3]:
df = pd.read_csv(raw_file)

In [4]:
df.columns =  ['timestamp', 'email', 'gender', 'location', 'is_international', 'ethnicity', 
         # Background
         'parents_edu', 'parents_technical', 'family_income', 'admission_avg', 'hs_extras',
         # Coding
         'code_start_age', 'fav_lang', 'text_editor', 'work_os', 'phone', 'num_hackathons', 'side_proj', 
         # Jobs
         'coop_name_1', 'coop_salary_1', 'coop_loc_1',
         'coop_name_2', 'coop_salary_2', 'coop_loc_2',
         'coop_name_3', 'coop_salary_3', 'coop_loc_3',
         'coop_name_4', 'coop_salary_4', 'coop_loc_4',
         'coop_name_5', 'coop_salary_5', 'coop_loc_5',
         'coop_name_6', 'coop_salary_6', 'coop_loc_6',
         'fav_coop', 'ft_status',
         # Lifestyle
         'exercise', 'cooking', 'sleep_time', 
         # School
         'uni_extras', 'is_se_orig', 'fav_course', 'least_fav_course', 'num_courses_failed', 
         'term_avg_1a', 'class_attendance_1a',
         'term_avg_1b', 'class_attendance_1b',
         'term_avg_2a', 'class_attendance_2a',
         'term_avg_2b', 'class_attendance_2b',
         'term_avg_3a', 'class_attendance_3a',
         'term_avg_3b', 'class_attendance_3b',
         'term_avg_4a', 'class_attendance_4a',
         'hardest_term', 'easiest_term', 'preferred_program', 'soft_eng_rating',
         # Future
         'preferred_company_size', 'work_loc', 'se_friendships', 'career_motives', 'grad_school', 'num_years_soft_eng',
         'preferred_tech_discipline', 'school_debt', 'se_advice'
        ]

In [74]:
def gender_map(row):
    new_gender = row['gender'].lowercase()

    if (new_gender.starts_with('Prefer not')):
        new_gender = 'undisclosed'

    return new_gender

def get_valid_coop_salaries(row):
    salaries = np.array(row[['coop_salary_1', 'coop_salary_2', 'coop_salary_3', 
                'coop_salary_4', 'coop_salary_5', 'coop_salary_6']], dtype=np.float32)
    return [max(0, min(100000, x)) for x in salaries]

def compute_coop_avg(row):
    return np.mean(get_valid_coop_salaries(row))

def compute_coop_median(row):
    return np.median(get_valid_coop_salaries(row))

city_regex = re.compile('(.*),.*')
def get_city(row):
    result = city_regex.match(row['location'])
    if result:
        return result.group(1).strip()
    return "?"

country_regex = re.compile('.*,(.*)')
def get_country(row):
    result = country_regex.match(row['location'])
    if result:
        return result.group(1).strip()
    return "?"

df['coop_salary_avg'] = df.apply(compute_coop_avg, axis=1)
df['coop_salary_median'] = df.apply(compute_coop_median, axis=1)
df['city_of_origin'] = df.apply(get_city, axis=1)
df['country_of_origin'] = df.apply(get_country, axis=1)

In [75]:
def get_valid_term_grade_avgs(row):
    avgs = np.array(row[['term_avg_1a', 'term_avg_1b', 'term_avg_2a', 'term_avg_2b'
                'term_avg_3a', 'term_avg_3b', 'term_avg_4a']], dtype=np.float32)
    return [max(0, min(100, x)) for x in avgs]

def compute_culm_grade_avg(row):
    return np.mean(get_valid_term_grade_avgs(row))

def compute_culm_grade_median(row):
    return np.median(get_valid_term_grade_avgs(row))

df['culm_grade_avg'] = df.apply(compute_culm_grade_avg, axis=1)
df['culm_grade_median'] = df.apply(compute_culm_grade_median, axis=1)

/Users/chris/Projects/se-2018-class-profile/env/lib/python2.7/site-packages/pandas/core/series.py:705: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [7]:
def round_admission_avg(x):
    x = min(100, max(70, x))
    return int(round(x))

df['rounded_admission_avg'] = df['admission_avg'].apply(round_admission_avg)

In [76]:
"""
Processing comma-separated lists, e.g. hs_extras, uni_extras.
"""
def to_lower(row, colname):
    return row[colname].lower()

def replace_strs(row, colname, transforms=[]):
    tmp = row
    
    for src_str, dst_str in transforms:
        tmp[colname] = tmp[colname].replace(src_str, dst_str)
    return tmp[colname]

def split_and_count(df, colname, delimiter=","):
    return pd.DataFrame([ x.strip() for x in df[colname].str.split(delimiter, expand=True).values.flatten() if x is not None ]).groupby(0).size()


extra_transforms = [
    ('hack the north', 'hackathon'),
    ('team', '')
]
df['uni_extras_normed'] = df.apply(partial(to_lower, colname='uni_extras'), axis=1)
df['uni_extras_normed'] = df.apply(partial(replace_strs, colname='uni_extras_normed', transforms=extra_transforms), axis=1)


In [80]:
split_and_count(df, 'uni_extras', ',')

0
Blueprint               2
Engsoc                  2
Hack the North          4
Mock Interviews         2
Residence Ambassador    2
Resume Critiques        2
SE community            4
UW/UX                   4
Ultimate Frisbee        2
Volunteering            2
dtype: int64

In [81]:
split_and_count(df, 'uni_extras_normed', ',')

0
blueprint               2
engsoc                  2
hackathon               4
mock interviews         2
residence ambassador    2
resume critiques        2
se community            4
ultimate frisbee        2
uw/ux                   4
volunteering            2
dtype: int64

In [9]:
df.to_csv(results_file)